In [36]:
%load_ext autoreload
%autoreload 2
from readers import GetProcessedInputData
from feature_normalization import NormalizationMethod


gt_list, train_signatures, test_signatures = \
  GetProcessedInputData(NormalizationMethod.NONE, is_test_mode=False)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
#For testing only.

train_signatures = train_signatures[:10]
test_signatures = test_signatures[:20]

In [38]:
from compute_distances import ComputeDistances

distances = ComputeDistances(train_signatures, test_signatures)

In [ ]:
print(distances)

In [40]:
print(distances)

{'023-02': {'026-41': 401.6983513375347, '011-06': 290.84020364042874, '016-39': 30.706745942462984, '008-16': 123.5349465310569, '027-05': 62.040912517166234, '022-17': 45.066648568303975, '013-13': 167.269408363153, '011-19': 297.29828177462133, '026-16': 390.644615438118, '017-28': 25.57680129649914, '025-28': 209.5407221309598, '016-22': 54.59116322617114, '021-16': 93.59332000739924, '008-08': 68.63260538575602, '008-12': 80.5514572174451, '030-21': 504.9249940524898, '001-08': 30.57523279333089, '019-07': 68.23691576925823, '016-26': 33.914997765074034, '007-42': 140.24432906608945}, '024-05': {'026-41': 257.00648107441975, '011-06': 634.5184147285355, '016-39': 88.19578119722833, '008-16': 131.0853611043492, '027-05': 187.08495432371208, '022-17': 162.53466440329595, '013-13': 325.85667764907475, '011-19': 298.12298608824733, '026-16': 637.2361954955464, '017-28': 92.3011602478313, '025-28': 460.2167057676388, '016-22': 116.5392497733064, '021-16': 98.76055935623604, '008-08': 1

In [ ]:
import pickle   
#Put distances dictionary into a pickle file
file = open("signatures.pkl", "wb")
pickle.dump(distances, file)
file.close()

In [ ]:
import pickle   
#Retrieve the distance dictionary from a pickle file
file_to_read = open("signatures.pkl", "rb")
distances = pickle.load(file_to_read)
file_to_read.close()

In [ ]:
# TODO: write output file sign-ver.csv (slide 14)
import csv
with open("sign-ver.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(outputData)